In [1]:
import pandas as pd
import dask.dataframe as dd
from dask.distributed import Client
import numpy as np

In [2]:
client = Client("tcp://172.31.24.178:8786")
client.restart()

<Client: 'tcp://172.31.24.178:8786' processes=0 threads=0, memory=0 B>

In [3]:
ddf = dd.read_csv("user_reviews_trimmed.csv")

In [4]:
ddf = ddf[["reviewerID", "helpful", "overall", "unixReviewTime"]] 

In [5]:
ddf["unixReviewTime"] = dd.to_datetime(ddf["unixReviewTime"].astype('float'), unit='s', origin='unix').dt.year

In [6]:
ddf.head()

,reviewerID,helpful,overall,unixReviewTime
0,A2T0RJ91B0PQ03,"[0, 0]",1.0,2014
1,A3TYW0XA8HSGWB,"[0, 0]",5.0,2014
2,A2CME0TQU2IVVB,"[1, 1]",5.0,2012
3,A2E5IDLX7R388S,"[0, 0]",5.0,2014
4,A3CIEMYUGV6ZMR,"[0, 0]",5.0,2012


In [7]:
%%time

ddf = ddf.set_index("reviewerID")

def get_helpful_votes(series):
    ans = 0
    for ele in series:
        ans += eval(ele)[0]
    return ans
    
def get_total_votes(series):
    ans = 0
    for ele in series:
        ans += eval(ele)[1]
    return ans

meta_df = pd.DataFrame(columns=[('overall', 'count'), ('overall', 'mean'), ('unixReviewTime', 'min'), ('helpful', 'get_helpful_votes'), ('helpful', 'get_total_votes')], dtype=int)

agg_fn = {'overall': ['count','mean'], 
          'unixReviewTime':'min', 
          'helpful':[get_helpful_votes,get_total_votes]
         }

ddf1 = ddf.map_partitions(lambda df:df.groupby('reviewerID').agg(agg_fn), meta=meta_df)

meta_df = pd.DataFrame(columns=[('overall', 'count'), ('overall', 'mean'), ('unixReviewTime', 'min'), ('helpful', 'get_helpful_votes'), ('helpful', 'get_total_votes')], dtype=int)

ddf2 = ddf1.rename(columns={('overall', 'count'): "number_products_rated", 
                                          ('overall', 'mean'): "avg_ratings",
                                          ('unixReviewTime', 'min'): "reviewing_since",
                                          ('helpful', 'get_helpful_votes'):"helpful_votes",
                                          ('helpful', 'get_total_votes'): "total_votes"
                                         })

print(f"ddf2 num partitons = {ddf2.npartitions}")

submit = ddf2.describe().round(2).compute() 


ddf2 num partitons = 53
CPU times: user 343 ms, sys: 14.3 ms, total: 358 ms
Wall time: 1min 39s


In [8]:
submit

,number_products_rated,avg_ratings,reviewing_since,helpful_votes,total_votes
count,3954028.00,3954028.00,3954028.00,3954028.00,3954028.00
mean,1.52,4.14,2011.52,3.13,4.22
std,3.14,1.24,3.06,33.00,36.68
min,1.00,1.00,1996.00,0.00,0.00
25%,1.00,4.00,2011.00,0.00,0.00
50%,1.00,5.00,2013.00,0.00,1.00
75%,1.00,5.00,2013.00,2.00,3.00
max,3169.00,5.00,2014.00,38198.00,38604.00
